# ZRP User Guide
The purpose of this notebook is to illustrate how to use ZRP, the main class of the zrp package that processes user input data &  returns race/ethnicity predictions

In [1]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi=False

In [2]:
from os.path import join, expanduser
import pandas as pd
import sys
import os
import re
import warnings

## Set source code path here

In [3]:
warnings.filterwarnings(action='once')
home = expanduser('~')

src_path = '{}/zest/zrp'.format(home)
sys.path.append(src_path)

In [4]:
from zrp import ZRP
from zrp.prepare.utils import load_file

/usr/local/lib/python3.9/site-packages/category_encoders/utils.py:5: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  from scipy.sparse.csr import csr_matrix
/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:35: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  version = LooseVersion(pd.__version__)
/usr/local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex
/usr/local/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Load sample data for prediction
Load list of New Jersey Mayors downloaded from https://www.nj.gov/dca/home/2022mayors.csv 

In [5]:
df = load_file(src_path + "/examples/2022-nj-mayors.csv")
df.shape

(565, 18)

In [6]:
df

,MUNI CODE,MUNI NAME,COUNTY,ADDRESS 1,ADDRESS 2,CITY,STATE,ZIP,PHONE,FAX,MAYOR NAME,TERM START,TERM END,FORM,TERM LEGNTH,EMAIL,SOCIAL MEDIA HANDLE,Municipal Contact List
0,1330,Aberdeen Township,Monmouth,One Aberdeen Square,NaN,Aberdeen,NJ,07747-2300,(732) 583-4200,NaN,Fred Tagliarini,NaN,12/31/2025,COUNCIL-MANAGER,4,fred.tagliarini@aberdeennj.org,NaN,NaN
1,0101,Absecon City,Atlantic,Absecon Municipal Complex,500 Mill Road,Absecon,NJ,08201,(609) 641-0663,(609) 645-5098,Kimberly Horton,NaN,12/31/2024,MAYOR-COUNCIL,3,khorton@abseconnj.org,NaN,NaN
2,1001,Alexandria Township,Hunterdon,782 Frenchtown Road,NaN,Milford,NJ,08848,(908) 996-7071,NaN,Gabe Plumer,NaN,12/31/2022,TOWNSHIP,3,clerk@alexandrianj.gov,NaN,NaN
3,2101,Allamuchy Township,Warren,Post Office Box A,NaN,Allamuchy,NJ,07820,(908) 852-5132,NaN,Rosemary Tuohy,NaN,12/31/2024,FAULKNER ACT,3,mayor@allamuchynj.org,NaN,NaN
4,0201,Allendale Borough,Bergen,500 West Crescent Avenue,NaN,Allendale,NJ,07401,(201) 818-4400,NaN,Ari Bernstein,NaN,12/31/2022,NaN,NaN,aribernstein@allendalenj.gov,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
560,0269,Wood-Ridge Borough,Bergen,85 Humboldt Street,NaN,Wood-Ridge,NJ,07075-2344,(201) 939-0202,NaN,Paul A Sarlo,NaN,12/31/2023,NaN,NaN,psarlo@njwoodridge.org,NaN,NaN
561,1715,Woodstown Borough,Salem,Post Office Box 286,NaN,Woodstown,NJ,08098,(856) 769-2200,NaN,Donald Dietrich,NaN,12/31/2023,NaN,NaN,Don.dietrich@comcast.net,NaN,NaN
562,0824,Woolwich Township,Gloucester,120 Village Green Drive,NaN,Woolwich Township,NJ,08085-3180,(856) 467-2666,NaN,Craig Frederick,NaN,12/31/2024,NaN,NaN,cfrederick@woolwichtwp.org,NaN,NaN
563,0340,Wrightstown Borough,Burlington,21 Saylors Pond Road,NaN,Wrightstown,NJ,08562,(609) 723-4450,(609) 723-7137,Donald Cottrell,NaN,12/31/2022,NaN,NaN,mayor@wrightstownborough.com,NaN,NaN


### sample data
sample for test case

In [9]:
samp = df.copy()
samp = samp[samp[key].isin(test_ids)]
samp.shape

(11, 14)

## ZRP   
- Processes input data
- Generates geo mappings
    - Requires standard address categories:
    - House Number
    - Street Address (including directional, street name, & street suffix)
    - City
    - State
    - Zip/postal code (prefer ZCTA5)
- Integrates processed data with American Community Survey (ACS) data
- Generates feature engineered data
- Predicts race & ethnicity

In [13]:
%%time
zest_race_predictor = ZRP()
zest_race_predictor.fit()
output = zest_race_predictor.transform(samp)

100%|██████████| 11/11 [00:00<00:00, 38035.73it/s]

Data is loaded
   Formatting P1
   Formatting P2
reduce whitespace

[Start] Preparing geo data
  The following states are included in the data: ['GA']
   ... on state: GA

   Data is loaded
   [Start] Processing geo data
/d/shared/zrp/shared_data
      ...address cleaning
      ...replicating address
         ...Base
         ...Map street suffixes...



[Parallel(n_jobs=49)]: Done  11 out of  11 | elapsed:    0.0s finished


         ...Mapped & split by street suffixes...
         ...Number processing...

     Address dataframe expansion is complete! (n=17)
      ...formatting
   [Completed] Processing geo data
   [Start] Mapping geo data
      ...merge user input & lookup table
      ...mapping


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


...

100%|██████████| 1/1 [00:00<00:00, 3155.98it/s]

App FE (in transform post data_fe 1) (12, 94)
App FE (in transform post data_fe 2) (12, 105)
App FE (end transform) (12, 105)
Custom Ratios (in transform) (12, 105)
Custom Ratios (end transform) (12, 126)
Name Aggregation (in transform) (12, 126)



[Parallel(n_jobs=90)]: Done   1 out of   1 | elapsed:    0.0s finished


(11, 15)
(11, 15)
Empty DataFrame
Columns: [HISPANIC_last_name, BLACK_middle_name, AAPI_middle_name, WHITE_last_name, BLACK_last_name, AAPI_last_name, AIAN_first_name, WHITE_middle_name, AAPI_first_name, HISPANIC_first_name, AIAN_middle_name, HISPANIC_middle_name, WHITE_first_name, AIAN_last_name, BLACK_first_name]
Index: []

(12, 126)

(11, 110)

(11, 15)

Output saved
CPU times: user 44.3 s, sys: 14.2 s, total: 58.5 s
Wall time: 27.9 s


/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/pandas/compat/_optional.py:116: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if distutils.version.LooseVersion(version) < minimum_version:
/home/kam/.conda/envs/zrp_q1_22/lib/python3.7/site-packages/setuptools/_distutils/version.py:351: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)


In [14]:
output.reset_index(drop=True)

,AAPI,AIAN,BLACK,HISPANIC,WHITE,source_block_group
0,0.003693,0.012285,0.689880,0.010139,0.284002,1
1,0.003317,0.008846,0.086520,0.006012,0.895306,1
2,0.015528,0.001163,0.001545,0.025489,0.956274,1
3,0.136450,0.035330,0.537617,0.071682,0.218921,1
4,0.009961,0.000892,0.948494,0.010603,0.030050,1
5,0.003140,0.000066,0.000432,0.994575,0.001788,1
6,0.003461,0.002152,0.013817,0.007392,0.973178,1
7,0.013305,0.010755,0.013567,0.024665,0.937708,1
8,0.013309,0.001035,0.003674,0.024191,0.957790,1
9,0.016148,0.000652,0.125726,0.012625,0.844849,1
